# Lesson 4: Geospatial Data and Mapping in Python

Today we're going to learn about using geospatial data in Python. Our focus will be on using the GeoPandas package: a package that handles geospatial in familiar Pandas objects like DataFrames and Series. 

We will learn how to pull geospatial data into our Python environment, create geographies, make geospatial joins, and map geospatial data. 

Geospatial data, sometimes called GIS data, is any data which is mapped onto the globe. The most common GIS data typically works with lattitudes and longitudes, but it can also include altitudes and sometimes other geographic information. Indeed, GIS stands for Geographic Information System, and so can generically refer to anything.

## Exercise 1
Let's start off by importing our typical suite of packages: Pandas, NumPy, and Matplotlib and setting our working directory.

## Exercise 2
Now let's try importing GeoPandas (geopandas) with the alias gpd.

You probably received a message saying that you don't have the module 'geopandas'. So, let's go ahead and install geopandas into our Python environment.

## Exercise 3
Use the following command to install GeoPandas into your Python environment: conda install -c conda-forge geopandas.

Now try importing GeoPandas again.

Great! Geospatial data comes in Shapefiles. GeoPandas will allow us to pull Shapefiles into our Python environment. However, Shapefiles aren't as simple as CSVs. Instead, they require other files to help define the geographic information it contains. Therefore, we have to download Shapefiles in ZIP folders containing all the different files we need.

## Exercise 4
Download the 2012 Ward data from the DC Open Data portal to your working directory, and extract all the files.

Now we'll use GeoPandas to pull the Shapefile into our notebook as a GeoDataFrame.

In [ ]:
wards = gpd.read_file('C:\\Users\\peter.casey\\Downloads\\Ward_from_2012\\Ward_from_2012.shp', crs='EPSG:4326')
wards.head().T

Because the Earth is a not-quite-sphere but our screens are flat, it is important to remember that working with geospatial data that it always consists of two parts: a projection (or coordinate reference system) and a collection of geometries.

The most common projection of geospatial data you'll see is WGS84 (World Geodetic System 1984) which affixes to every point on the Earth a latitude an longitude. In other words, it is a Mercator projection. EPSG:4326 is WSG84 in EPSG language. The vast majority (all?) of DC data is based on the Master Address Repository (or MAR), which lists things in latitudes and longitudes in the WGS84 projection. Be warned when working with these sorts of files that you will potentially run into issues near the poles with Mercator projections, but they should be generally fine around DC.

As for geometries, there are essentially three major building blocks: points, lines, and polygons. All GIS operations are defined for all three types (as well as combinations of all three types), but be warned that the behavior of, say, asking for the centroid of a point and a polygon may not behave as you expect. Each GeoPandas dataframe has a geometry that is either a polygon, line, or point.

Let's take a look at the geometry column in our GeoPandas dataframe of the District's wards.

In [ ]:
wards.geometry

Now let's plot the geometries.

In [ ]:
wards.geometry.plot()

We can also create geometries from Pandas dataframes that have longitude and latitude. Let's do this with our 311 data from 2016.

## Exercise 5
Import the city service requests from 2016 that we've been using.

There are lots of service requests in the city each year, so let's focus in on just one. Perhaps one in which I have a little expertise...

## Exercise 6
Create a new dataframe called 'rats' from just the city service requests with the service code description 'Rodent Inspection and Treatment' from August 2016.

We can use the longitude and latitude columns to create a geometry for this data. 

For this we need to import Point function from the shapely package.

In [ ]:
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(rats.LONGITUDE.apply(float), rats.LATITUDE.apply(float))]

We now have a geometry, in this case points. But recall that you also need a projection, or crs. If we want to match these points to our ward polygons, they need to have the same coordinate system. 

In [ ]:
crs = {'init': 'epsg:4326'}

Now that we have a geometry and a projection, we can combine them with the original Pandas dataframe into a new GeoPandas dataframe.

In [ ]:
points = gpd.GeoDataFrame(rats, crs=crs, geometry=geometry)

Great! Now we have a new GeoPandas dataframe with points for each of the requests for rodent inspection and treatment in 2016.

We can now use this to map them on to DC. Let's try it out! 

We'll just look at requests for rodent inspection and treatment in August so the plot doesn't get too busy. 

In [ ]:
points['Date'] = pd.to_datetime(rats['SERVICEORDERDATE'])
points[points.Date.dt.month==8].geometry.plot()

Neat! We can kind of see the outline of the District, but it would be even better to plot it ONTO the wards. Let's try that out.

In [ ]:
fig, ax = plt.subplots()
wards.plot(ax=ax)
points[points.Date.dt.month==8].plot(ax=ax, color='black')

Ugh! What's with the weird shape? Turns out we have to fix the figure's aspect ratio.

In [ ]:
fig, ax = plt.subplots()
wards.plot(ax=ax)
points[points.Date.dt.month==8].plot(ax=ax, color='black')
ax.set_aspect('equal')

Much better but we can improve the map some by making it bigger, making the polygons white and the points translucent.

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
wards.plot(ax=ax, color='white')
points[points.Date.dt.month==8].plot(ax=ax, color='blue', alpha=0.2)

Now we're talking! But what if we want to get the number of requests for rodent inspection and treatment in each ward? And what if want to map the wards based on such an aggregate?

We can also use SPATIAL JOINS to join the points we created to our ward polygons.

Joins and merges are something we'll be doing a LOT of over the course of the semester. These are crucial for pulling together data from disparate sources. Often, joins and merges are based on the values of a column that two dataframes have in common. In fact, there's an "easy way" that we could combine this data by merging based on the WARD in each dataframe using the Pandas merge function.

In [ ]:
ward_rats = wards.merge(points, how='left', on='WARD')
ward_rats.head().T

Our data is merged, but there are problems with such a merge. Such a merge creates a Pandas dataframe rather than a GeoPandas dataframe, so it does not preserve our geometry. We can recreate the geometry, but for the sake of this lesson, let's focus instead on the spatial join, since this will be necessary when a common column like 'WARD' is not available.

In [ ]:
geo_rats = gpd.sjoin(wards, points, how='left', op='intersects')

Let's dissect this. The 'how' parameter refers to the type of join. A 'left' join preserves the rows on the left. For example, if there were no requests for rodent inspection and treatment in Ward 7, this join would still include Ward 7 in the list of wards. The 'op' command tells us what relationship to look for between the geometries of the two GeoPandas dataframes. It does not matter what 'op' we choose for a points-to-polygons join, but it will matter when we join polygons. 

Now, let's take a look at the geometry of the new dataframe.

In [ ]:
geo_rats.geometry.head()

It's a polygon! Let's map it!

In [ ]:
geo_rats.geometry.plot()

Oh no! That doesn't help us at all! How do we create a useful plot?

First, let's create an informative aggregate, like the percent of rat calls citywide that are in each ward. We'll do this by get a count of the number of requests in each ward and then dividing it by the total number of requests.

In [ ]:
geo_rats.WARD.value_counts/rats.shape[0]

Oops! What went wrong?

Take a look at the columns. When we do a join and the two dataframes each has a column of the same name (such as "WARD"), then the function will add a suffix to each. In this case, "left" and "right". Let's try it again. 

In [ ]:
geo_rats.WARD_left.value_counts()/geo_rats.shape[0]

As I mentioned before, this provides us with an index and a column of values, so we can actually make this into a DataFrame.

In [ ]:
rats_pcttot = pd.DataFrame(geo_rats.WARD_left.value_counts()/geo_rats.shape[0])
rats_pcttot.columns = ['rats_pcttot']
rats_pcttot

Now we can merge this to the wards GeoDataFrame and map it.

In [ ]:
ward_rats = wards.merge(rats_pcttot, how="left", left_on='WARD', right_index=True)
ward_rats.head().T

Now we can create a heat map.

In [ ]:
ward_rats.plot(column='rats_pcttot', cmap='YlOrRd')

We can also create a color bar to help us interpret the heatmap.

In [ ]:
vmin = ward_rats.rats_pcttot.min()
vmax = ward_rats.rats_pcttot.max()

ax = ward_rats.plot('rats_pcttot', cmap='YlOrRd', vmin=vmin, vmax=vmax)

fig = ax.get_figure()
cax = fig.add_axes([0.9, 0.1, 0.03, 0.8])
sm = plt.cm.ScalarMappable(cmap='YlOrRd', norm=plt.Normalize(vmin=vmin, vmax=vmax))
# fake up the array of the scalar mappable. Urgh...
sm._A = []
fig.colorbar(sm, cax=cax)

## Exercise 7
Now that we've shown the distribution of requests for rodent inspection and treatment across wards, let's look at Advisory Neighborhood Commissions, or ANCs. Each of DC's seven wards is divided into ANCs that deal with neighborhood needs at an even more local levels. Find the 2013 ANC's on the Open Data Portal, download and extract the Shapefile and pull it into the notebook as a GeoPandas dataframe. Then join city service requests to this GeoDataFrame. You may use whichever requests you'd like to explore. Create aggregates and maps like the ones we created above, and let's share them in class. Enjoy!